In [ ]:
import os

# Path to the datasets folder
datasets_path = './datasets'

# Read and display each txt file separately
if os.path.exists(datasets_path):
    for filename in os.listdir(datasets_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(datasets_path, filename)
            with open(file_path, 'r') as file:
                content = file.read()
                print(f"Contents of {filename}:\n{content}\n{'-'*40}\n")
else:
    print("The datasets folder does not exist or no .txt files were found.")


In [ ]:
# Instale o PyCaret (caso ainda não tenha feito isso)
!pip install pycaret




In [ ]:
import os
import pandas as pd
from pycaret.regression import setup, compare_models

# Caminho para a pasta datasets
datasets_path = './datasets'
output_file = 'melhores_modelos.txt'

# Limpar o conteúdo anterior do arquivo de saída
with open(output_file, 'w') as f:
    f.write("Melhores Modelos para Cada Dataset\n")
    f.write("="*40 + "\n\n")

# Verificar se a pasta existe e ler todos os arquivos txt
if os.path.exists(datasets_path):
    for filename in os.listdir(datasets_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(datasets_path, filename)
            
            # Carregar os dados
            try:
                data = pd.read_csv(file_path)
                print(f"Lendo e processando: {filename}")
                print(f"Colunas disponíveis no arquivo {filename}: {data.columns.tolist()}\n")
                
                # Usar a última coluna como variável alvo
                target_column = data.columns[-1]
                print(f"Usando a coluna '{target_column}' como variável alvo.\n")

                # Configuração do PyCaret para o problema de regressão
                reg = setup(data, target=target_column)
                
                # Comparar todos os modelos de regressão
                best_model = compare_models()
                print(f"Melhor modelo para {filename}: {best_model}\n{'-'*40}\n")
                
                # Salvar o melhor modelo no arquivo
                with open(output_file, 'a') as f:
                    f.write(f"Arquivo: {filename}\n")
                    f.write(f"Melhor Modelo: {best_model}\n")
                    f.write("-"*40 + "\n\n")
            
            except Exception as e:
                print(f"Erro ao processar {filename}: {e}\n")
else:
    print("A pasta 'datasets' não existe ou está vazia.")
    
print(f"Os melhores modelos foram salvos em {output_file}.")


In [ ]:
import os
import pandas as pd
from pycaret.regression import setup, create_model, stack_models, pull

# Caminho para a pasta datasets
datasets_path = './datasets'

# Melhores modelos escolhidos manualmente para cada dataset
best_models = {
    "cleaned_cocomo81.txt": "dt",  # DecisionTreeRegressor
    "cleaned_desharnais.txt": "huber",  # HuberRegressor
    "cleaned_isbsg.txt": "lr",  # LinearRegression
    "cleaned_maxwell.txt": "catboost",  # CatBoostRegressor
    "cleaned_china.txt": "lr"  # LinearRegression
}

# Lista para armazenar os resultados
results_list = []

# Verificar se a pasta existe e ler todos os arquivos txt
if os.path.exists(datasets_path):
    for filename in os.listdir(datasets_path):
        if filename in best_models:
            file_path = os.path.join(datasets_path, filename)
            
            try:
                # Carregar os dados
                data = pd.read_csv(file_path)
                print(f"\nLendo e processando: {filename}")
                
                # Usar a última coluna como variável alvo
                target_column = data.columns[-1]
                print(f"Usando a coluna '{target_column}' como variável alvo.\n")

                # Configuração do PyCaret para o problema de regressão
                reg = setup(data, target=target_column, verbose=False)
                
                # Criar os modelos individuais
                model1 = create_model(best_models[filename])
                
                # Adicionar mais modelos conforme necessário e criar um ensemble
                if best_models[filename] == "lr":
                    model2 = create_model("dt")  # Adicionando um Decision Tree como exemplo
                else:
                    model2 = create_model("lr")  # Adicionando uma Regressão Linear como exemplo
                
                # Criar o ensemble usando a abordagem de empilhamento
                ensemble_model = stack_models([model1, model2])
                
                # Obter o DataFrame de métricas de desempenho
                metrics_df = pull()
                
                # Adicionar o nome do arquivo ao DataFrame
                metrics_df['Dataset'] = filename
                
                # Armazenar os resultados
                results_list.append(metrics_df)
                
                print(f"Modelo Ensemble criado para {filename}\n{'-'*40}\n")

            except Exception as e:
                print(f"Erro ao processar {filename}: {e}\n")

# Concatenar todos os DataFrames em um único DataFrame
if results_list:
    final_results_df = pd.concat(results_list)
    # Reorganizar as colunas para mostrar o nome do dataset primeiro
    final_results_df = final_results_df[['Dataset'] + [col for col in final_results_df.columns if col != 'Dataset']]
    
    # Salvar a tabela de resultados em um arquivo TXT
    output_file_path = 'ensemble_results.txt'
    with open(output_file_path, 'w') as file:
        file.write("Tabela de Resultados:\n")
        file.write(final_results_df.to_string(index=False))
    
    print(f"\nTabela de Resultados salva em: {output_file_path}")
else:
    print("Nenhum resultado foi gerado.")
